# Natural language to SQL

**Run in [Google Colab](https://colab.research.google.com/) For GPU.**

This model have  Mistral as a base and it has been fine-tuned to excel in SQL code generation.

In [63]:
# from google.colab import userdata
# userdata.get('HF_TOKEN')

In [4]:
#Install the lastest versions of peft & transformers library recommended
#if you want to work with the most recent models
!pip install -q git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install bitsandbytes

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.4 MB/s eta 0:00:00
  Cloning https://github.com/huggingface/accelerate.git to /tmp/pip-

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import accelerate

In [6]:
model_name = "defog/sqlcoder-7b"

We need to create the Quantization configuration to load the Model.

It is a large model and I want it to fit in a 16GB GPU, I'm going to use a 4 bits quantization.

If you want to learn more about quantization, refer to this article: [QLoRA: Training a Large Language Model on a 16GB GPU.](https://medium.com/towards-artificial-intelligence/qlora-training-a-large-language-model-on-a-16gb-gpu-00ea965667c1)

You can try to use this model in a 8 bit quantizations and check in you see any improvements in the results.

In [7]:
bnb_config = BitsAndBytesConfig(
  load_in_4bit=True,
  bnb_4bit_use_double_quant=True,
  bnb_4bit_quant_type="nf4",
  bnb_4bit_compute_dtype=torch.bfloat16
)


To load the model I pass to the AutoModelForCasualLM teh quantization configurations, and HuggingFace take care of all the hard work.

In [8]:
foundation_model = AutoModelForCausalLM.from_pretrained(model_name,
                    quantization_config=bnb_config,
                    device_map='auto',
                    use_cache = True)

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
eos_token_id = tokenizer.convert_tokens_to_ids(["```"])[0]

tokenizer_config.json:   0%|          | 0.00/915 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

This function wraps the call to *model.generate*

In [10]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=400):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        num_return_sequences=1,
        eos_token_id=eos_token_id,
        pad_token_id=eos_token_id,
        max_new_tokens=max_new_tokens,
        do_sample=False,
        num_beams=5
    )
    return outputs

# Prompt without Shots.
In this first PROMPT we are going to give Instructions to the model and pass the structure of the Database.

The instructions are significantly different from those we are passing to GPT-3.5-Turbo. This model is really well fine-tuned, but it is smaller than GPT-3.5.

We need to be more clear with the instructions, as it does not have the same capacity to understand our orders as GPT-3.5.

In [11]:
sp_nl2sql = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE employees (
  ID_usr INTEGER,
  name TEXT
);

CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary FLOAT
);

CREATE TABLE studies (
  ID INTEGER,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
);


    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """

In [12]:
sp_nl2sql = sp_nl2sql.format(question="Return The name of the best paid employee")
print(sp_nl2sql)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question

    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

    CREATE TABLE employees (
  ID_usr INTEGER,
  name TEXT
);

CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary FLOAT
);

CREATE TABLE studies (
  ID INTEGER,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
);


    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `Return The name of the best paid employee`:
    ```sql3
    


In [13]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [14]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [15]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.id_usr = salary.id_usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;


The SQL Order is correct.

#Prompt with shots OpenAI Style.
In this second prompt we are going to add some Shots with samples to see if our SQL style affects the model.

In [16]:
sp_nl2sql2 = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

  CREATE TABLE employees (
  ID_usr INTEGER,
  name TEXT
);

CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary FLOAT
);

CREATE TABLE studies (
  ID INTEGER,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
);

    ### Response
Question: What is the average salary per institution?
SQL:
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN salary sa ON st.ID_usr = sa.ID_usr
GROUP BY st.Institution;

Question: List all employees and their salary.
SQL:
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr;

    `{question}`:
    ```sql3
    """


In [17]:
sp_nl2sql2 = sp_nl2sql2.format(question="Return The name of the best paid employee")
(print(sp_nl2sql2))


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to clearn more about teh Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

  CREATE TABLE employees (
  ID_usr INTEGER,
  name TEXT
);

CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary FLOAT
);

CREATE TABLE studies (
  ID INTEGER,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
);

    ### Response
Question: What is the average salary per institution?
SQL:
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN salary sa ON st.ID_usr = sa.ID_usr
GROUP BY st.Institution;

Question: List all employe

In [18]:
input_sentences = tokenizer(sp_nl2sql2, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [19]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name, MAX(s.salary) AS max_salary
     FROM employees e
     JOIN salary s ON e.ID_usr = s.ID_usr
     GROUP BY e.name
     ORDER BY max_salary DESC NULLS LAST
     LIMIT 1;


The Order is really different from the one obtained with the first prompt.

The first difference is the format. But The SQL is realy more simple, at least it is my sensation.

#Prompt with Shots in Sample Style.

In this prompt, we will place the examples in a separate section, and in the instructions, we will instruct the model to pay attention to them in order to generate the SQL commands.

In [20]:
sp_nl2sql3b = """
    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

  CREATE TABLE employees (
  ID_usr INTEGER,
  name TEXT
);

CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary FLOAT
);

CREATE TABLE studies (
  ID INTEGER,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
);

    ### Samples

Question: What is the average salary per institution?
SQL:
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN salary sa ON st.ID_usr = sa.ID_usr
GROUP BY st.Institution;

Question: List all employees and their salary.
SQL:
SELECT e.name, s.salary
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr;

    ### Response
    Based on your instructions, here is the SQL query I have generated to answer the question
    `{question}`:
    ```sql3
    """


In [21]:
sp_nl2sql3 = sp_nl2sql3b.format(question="Return The name of the best paid employee")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

  CREATE TABLE employees (
  ID_usr INTEGER,
  name TEXT
);

CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary FLOAT
);

CREATE TABLE studies (
  ID INTEGER,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
);

    ### Samples

Question: What is the average salary per institution?
SQL:
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN salary sa ON st.ID_usr = sa.ID_usr
GROUP BY st.Institution;

Question: List all employee

In [22]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [23]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name, MAX(s.salary) AS max_salary
     FROM employees e
     JOIN salary s ON e.ID_usr = s.ID_usr
     GROUP BY e.name
     ORDER BY max_salary DESC
     LIMIT 1;


#Now the question in Arabic.


In [24]:
sp_nl2sql3 = sp_nl2sql3b.format(question="إرجاع اسم الموظف الأعلى أجراً")
print (sp_nl2sql3)


    ### Instructions:
Your task is convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question
- **Use the samples SQL In the ### Samples section to learn more about the Databases structure


    ### Input
    Generate a SQL query that answers the question below.
    This query will run on a database whose schema is represented in this string:

  CREATE TABLE employees (
  ID_usr INTEGER,
  name TEXT
);

CREATE TABLE salary (
  ID_usr INTEGER,
  year DATE,
  salary FLOAT
);

CREATE TABLE studies (
  ID INTEGER,
  ID_usr INTEGER,
  educational_level INTEGER,
  Institution TEXT,
  Years DATE,
  Speciality TEXT
);

    ### Samples

Question: What is the average salary per institution?
SQL:
SELECT st.Institution, AVG(sa.salary) AS avg_salary
FROM studies st
JOIN salary sa ON st.ID_usr = sa.ID_usr
GROUP BY st.Institution;

Question: List all employee

In [25]:
input_sentences = tokenizer(sp_nl2sql3, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)
torch.cuda.empty_cache()

In [26]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT e.name, MAX(s.salary) AS max_salary
     FROM employees e
     JOIN salary s ON e.ID_usr = s.ID_usr
     GROUP BY e.name
     ORDER BY max_salary DESC
     LIMIT 1;


The generated SQL command is the same regardless of where we have placed the examples.

#Conclusions.

Let's see the three SQL's together.

* SELECT employees.name, MAX(salary.salary) AS max_salary FROM employees JOIN salary ON employees.ID_Usr = salary.ID_Usr GROUP BY employees.name ORDER BY max_salary DESC NULLS LAST LIMIT 1;

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* SELECT e.name
    FROM employees e
    JOIN salary s ON e.ID_Usr = s.ID_usr
    WHERE s.salary = (SELECT MAX(salary) FROM salary);

* Spanish Question: SELECT e.name
     FROM employees e
     JOIN salary s ON e.ID_Usr = s.ID_Usr
     WHERE s.salary = (SELECT MAX(salary) FROM salary)
     GROUP BY e.name
     ORDER BY COUNT(studies.ID_study) DESC
     LIMIT 1;


**The model has demonstrated that it is highly efficient in crafting SQL.** Additionally, it pays a lot of attention, perhaps too much, to the examples we provide. Clearly, these examples should be crafted by one of the best SQL programmers we have access to, though their use may not be essential.

On the other hand, although the model is clearly very proficient in SQL generation, during the creation of the notebook, I have encountered several issues because the commands need to be extremely clear. It doesn't handle typos well (which should not exist).

It appears to have some issues when it receives commands in Spanish. I assume this problem would be present in any language other than English. Therefore, since it's a tool that could be used by non-technical personnel, this should be considered in environments where English is not the primary language.

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [37]:
sp_nl2sql = """
### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question.

### Input:
Generate a SQL query that answers the question below.
This query will run on a database whose schema is represented in this string:

CREATE TABLE books (
  id INTEGER PRIMARY KEY,
  title TEXT,
  author_id INTEGER,
  genre TEXT,
  published_year INTEGER
);

CREATE TABLE authors (
  id INTEGER PRIMARY KEY,
  name TEXT,
  country TEXT
);

CREATE TABLE borrowings (
  id INTEGER PRIMARY KEY,
  book_id INTEGER,
  user_id INTEGER,
  borrowed_date DATE,
  return_date DATE
);

CREATE TABLE users (
  id INTEGER PRIMARY KEY,
  name TEXT,
  membership_date DATE
);

### Response
Based on your instructions, here is the SQL query I have generated to answer the question
`{question}`:
```sql3
"""


In [38]:
sp_nl2sql = sp_nl2sql.format(question="List all books published after 2015")
print(sp_nl2sql)


### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- **Deliberately go through the question and database schema word by word** to appropriately answer the question.

### Input:
Generate a SQL query that answers the question below.
This query will run on a database whose schema is represented in this string:

CREATE TABLE books (
  id INTEGER PRIMARY KEY,
  title TEXT,
  author_id INTEGER,
  genre TEXT,
  published_year INTEGER
);

CREATE TABLE authors (
  id INTEGER PRIMARY KEY,
  name TEXT,
  country TEXT
);

CREATE TABLE borrowings (
  id INTEGER PRIMARY KEY,
  book_id INTEGER,
  user_id INTEGER,
  borrowed_date DATE,
  return_date DATE
);

CREATE TABLE users (
  id INTEGER PRIMARY KEY,
  name TEXT,
  membership_date DATE
);

### Response
Based on your instructions, here is the SQL query I have generated to answer the question
`List all books published after 2015`:
```sql3



In [39]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [40]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [41]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT books.id, books.title, books.author_id, authors.name, books.genre, books.published_year FROM books JOIN authors ON books.author_id = authors.id WHERE books.published_year > 2015 ORDER BY books.title NULLS LAST;


In [42]:
sp_nl2sql = """
### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- Learn from the examples shown below.

### Input:
This query will run on a database whose schema is represented in this string:

CREATE TABLE books (
  id INTEGER PRIMARY KEY,
  title TEXT,
  author_id INTEGER,
  genre TEXT,
  published_year INTEGER
);

CREATE TABLE authors (
  id INTEGER PRIMARY KEY,
  name TEXT,
  country TEXT
);

CREATE TABLE borrowings (
  id INTEGER PRIMARY KEY,
  book_id INTEGER,
  user_id INTEGER,
  borrowed_date DATE,
  return_date DATE
);

CREATE TABLE users (
  id INTEGER PRIMARY KEY,
  name TEXT,
  membership_date DATE
);

### Response:
Question: Show all users who borrowed books in 2023.
SQL:
SELECT u.name
FROM users u
JOIN borrowings b ON u.id = b.user_id
WHERE EXTRACT(YEAR FROM b.borrowed_date) = 2023;

Question: List the names and countries of all authors who wrote science fiction books.
SQL:
SELECT a.name, a.country
FROM authors a
JOIN books b ON a.id = b.author_id
WHERE b.genre = 'Science Fiction';

Based on your instructions, here is the SQL query I have generated to answer the question "{question}":
```sql3
"""


In [43]:
sp_nl2sql = sp_nl2sql.format(question="List all users who borrowed books in 2023")
print(sp_nl2sql)


### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Adhere to these rules:
- Learn from the examples shown below.

### Input:
This query will run on a database whose schema is represented in this string:

CREATE TABLE books (
  id INTEGER PRIMARY KEY,
  title TEXT,
  author_id INTEGER,
  genre TEXT,
  published_year INTEGER
);

CREATE TABLE authors (
  id INTEGER PRIMARY KEY,
  name TEXT,
  country TEXT
);

CREATE TABLE borrowings (
  id INTEGER PRIMARY KEY,
  book_id INTEGER,
  user_id INTEGER,
  borrowed_date DATE,
  return_date DATE
);

CREATE TABLE users (
  id INTEGER PRIMARY KEY,
  name TEXT,
  membership_date DATE
);

### Response:
Question: Show all users who borrowed books in 2023.
SQL:
SELECT u.name
FROM users u
JOIN borrowings b ON u.id = b.user_id
WHERE EXTRACT(YEAR FROM b.borrowed_date) = 2023;

Question: List the names and countries of all authors who wrote science fiction books.
SQL:
SELECT a.name, a.country
FROM authors a

In [44]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [45]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [46]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT users.name FROM users JOIN borrowings ON users.id = borrowings.user_id WHERE EXTRACT(YEAR FROM borrowings.borrowed_date) = 2023;


In [57]:
sp_nl2sql = """
### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Use the examples in the Samples section to learn how to handle the structure and generate accurate queries.

### Input:
Database schema:

CREATE TABLE books (
  id INTEGER PRIMARY KEY,
  title TEXT,
  author_id INTEGER,
  genre TEXT,
  published_year INTEGER
);

CREATE TABLE authors (
  id INTEGER PRIMARY KEY,
  name TEXT,
  country TEXT
);

CREATE TABLE borrowings (
  id INTEGER PRIMARY KEY,
  book_id INTEGER,
  user_id INTEGER,
  borrowed_date DATE,
  return_date DATE
);

CREATE TABLE users (
  id INTEGER PRIMARY KEY,
  name TEXT,
  membership_date DATE
);

### Samples:
Question: Show all users who borrowed books in 2023.
SQL:
SELECT u.name
FROM users u
JOIN borrowings b ON u.id = b.user_id
WHERE EXTRACT(YEAR FROM b.borrowed_date) = 2023;

Question: List the names and countries of all authors who wrote science fiction books.
SQL:
SELECT a.name, a.country
FROM authors a
JOIN books b ON a.id = b.author_id
WHERE b.genre = 'Science Fiction';

Question: Show all books published in 2020.
SQL:
SELECT title FROM books WHERE published_year = 2020;

### Response:
Based on your instructions, here is the SQL query I have generated to answer the question "{question}":
```sql3
"""

In [59]:
sp_nl2sql = sp_nl2sql.format(question="Return titles of books written by authors from France")
print(sp_nl2sql)


### Instructions:
Your task is to convert a question into a SQL query, given a SQL database schema.
Use the examples in the Samples section to learn how to handle the structure and generate accurate queries.

### Input:
Database schema:

CREATE TABLE books (
  id INTEGER PRIMARY KEY,
  title TEXT,
  author_id INTEGER,
  genre TEXT,
  published_year INTEGER
);

CREATE TABLE authors (
  id INTEGER PRIMARY KEY,
  name TEXT,
  country TEXT
);

CREATE TABLE borrowings (
  id INTEGER PRIMARY KEY,
  book_id INTEGER,
  user_id INTEGER,
  borrowed_date DATE,
  return_date DATE
);

CREATE TABLE users (
  id INTEGER PRIMARY KEY,
  name TEXT,
  membership_date DATE
);

### Samples:
Question: Show all users who borrowed books in 2023.
SQL:
SELECT u.name
FROM users u
JOIN borrowings b ON u.id = b.user_id
WHERE EXTRACT(YEAR FROM b.borrowed_date) = 2023;

Question: List the names and countries of all authors who wrote science fiction books.
SQL:
SELECT a.name, a.country
FROM authors a
JOIN books b ON

In [60]:
input_sentences = tokenizer(sp_nl2sql, return_tensors="pt").to('cuda')
response = get_outputs(foundation_model, input_sentences, max_new_tokens=400)
SQL = tokenizer.batch_decode(response, skip_special_tokens=True)

In [61]:
#Empty the cache in orde to do more calls without problems.
torch.cuda.empty_cache()

In [62]:
print(SQL[0].split("```sql3")[-1].split("```")[0].split(";")[0].strip() + ";")

SELECT b.title FROM books b JOIN authors a ON b.author_id = a.id WHERE a.country = 'France';


# Prompt Evaluation Report – Natural Language to SQL  
**Database: Library System**

---

## Prompt 1 – Zero-shot (No Examples)

- **Type:** No examples provided  
- **Question:** List all books published after 2015  
- **Result:** Correct  
- **Notes:** The query was valid and matched the question.

---

## Prompt 2 – Few-shot (Examples in Response)

- **Type:** Examples written before the question  
- **Question:** List all users who borrowed books in 2023  
- **Result:** Correct  
- **Notes:** The model followed the structure of the examples accurately.

---

## Prompt 3 – Few-shot (Samples Section)

- **Type:** Examples placed in a separate Samples section  
- **Question:** Return titles of books written by authors from France  
- **Result:** Correct  
- **Notes:** The query was accurate and matched the expected output.

---